In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 7.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 880 kB 71.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=76a6c0365a6dbdbb060f9386e7e6eaf7b195577bbf0ae407094fc8001e06df4b
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=9207e9681c0eeeca629e217b347caf0f91456469689c6595bb9cbf2dd3b9faaf
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words
Looking in indexes: https://pypi.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [5]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoderSimple
from src.dataset.ldabert_2 import LDABERT2Dataset
import transformers
import sentence_transformers

from utils.metrics import windowdiff, pk

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

In [6]:
tf.__version__, transformers.__version__, sentence_transformers.__version__

('2.9.2', '4.3.3', '0.4.1')

## LDA BERT 2 Dataset

In [7]:
dataset_type = "clinical"
print("initializing testing dataset...")
testing_dataset = LDABERT2Dataset(dataset_type=dataset_type,
                                  dataset_slice="testing",
                                  pct_data=1,
                                  max_seq_length=512,
                                  max_segment_length=300,
                                  augment_pct=0,
                                  split="test",
                                  artificial_segments=False)

# process testing dataset
print("processing testing dataset...")
testing_sentences, testing_tokenized_sentences, testing_labels = testing_dataset.process()

vectors_filename = '{}_{}_testing_data.pkl'.format(testing_dataset.pct_data, testing_dataset.augment_pct)

testing_saved_vectors, testing_saved_labels, testing_saved_sentences, testing_saved_tokenized_sentences = testing_dataset.get_saved_vectors("test", testing_dataset.dataset_type, vectors_filename)

if len(testing_saved_vectors) == 0:
    testing_saved_vectors, testing_saved_labels, testing_saved_sentences, testing_saved_tokenized_sentences = testing_dataset.create_vectors("test", testing_dataset.dataset_type, vectors_filename)

testing_left_input, testing_mid_input, testing_right_input = testing_dataset.format_sentences_tri_input_plus(testing_saved_tokenized_sentences)
testing_lda_left_input, testing_lda_mid_input, testing_lda_right_input = testing_dataset.format_sentences_tri_input(testing_saved_vectors)

initializing testing dataset...
processing testing dataset...
artificial segments False


In [8]:
left_input, mid_input, right_input = testing_dataset.format_sentences_tri_input_plus(testing_saved_tokenized_sentences)

In [9]:
lda_left_input, lda_mid_input, lda_right_input = testing_dataset.format_sentences_tri_input(testing_saved_vectors)

In [10]:
left_input, mid_input, right_input = testing_dataset.format_sentences_tri_input_plus(testing_saved_tokenized_sentences)

## Model

In [11]:
lda_gamma = 15
model = ContextEncoderSimple(final_dropout=0.5,
                            dense_neurons=128,
                            max_sentence_length=128,
                            bert_trainable=False,
                            gamma=lda_gamma)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
# to test
# # = to test
# ## = tested

# checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/clinical-44979-1-pct-1-aug_oGsMF/no-finetune/checkpoint.ckpt' # 150 epochs, with GSSD
## checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/clinical-24771-1-pct-0-aug_eI2us/no-finetune/checkpoint.ckpt' # 200 epochs, no GSSD
# checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/clinical-24771-1-pct-0-aug_du7Ee/finetune/checkpoint.ckpt' # 5 + 50 epochs, no GSSD

## checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/wiki-40561-1-pct-0-aug_SwPSB/no-finetune/checkpoint.ckpt' # 200 epochs, no GSSD
# checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/wiki-40561-1-pct-0-aug_jA6HL/finetune/checkpoint.ckpt' # 5 + 50 epochs, no GSSD

## checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/fiction-43721-1-pct-0-aug_BzsWJ/no-finetune/checkpoint.ckpt' # 200 epochs, no GSSD
# checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/fiction-43721-1-pct-0-aug_wmZJ5/finetune/checkpoint.ckpt' # 5 + 15 epochs, no GSSD
# checkpoint_filepath = config.root_path+'/models/LDABERT2/simple/fiction-43721-1-pct-0-aug_DRh4S/finetune/checkpoint.ckpt' # 5 + 50 epochs, no GSSD

In [16]:
hash = "du7Ee"
dataset_type = "clinical"
num_samples = 24771
augmentation = False
finetune = True

# model_path = 'LDABERT2/hybrid-conv/{}-{}-1-pct-1-aug_{}/finetune-True'.format(dataset_type, num_samples, hash)
model_path = 'LDABERT2/simple/{}-{}-1-pct-{}-aug_{}/{}finetune'.format(dataset_type, num_samples, (1 if augmentation==True else 0), hash, ('' if finetune==True else 'no-'))
checkpoint_filepath = config.root_path+'/models/{}/checkpoint.ckpt'.format(model_path)
log_file = '{}/predictions/{}/predictions.csv'.format(config.root_path, hash)

print(checkpoint_filepath)

/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/simple/clinical-24771-1-pct-0-aug_du7Ee/finetune/checkpoint.ckpt


In [17]:
# try:
model.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

Prepare the checkpoints

In [28]:
checkpoints = []

In [29]:
hash = "du7Ee"
dataset_type = "clinical"
num_samples = 24771
augmentation = False
finetune = True

# model_path = 'LDABERT2/hybrid-conv/{}-{}-1-pct-1-aug_{}/finetune-True'.format(dataset_type, num_samples, hash)
model_path = 'LDABERT2/simple/{}-{}-1-pct-{}-aug_{}/{}finetune'.format(dataset_type, num_samples, (1 if augmentation==True else 0), hash, ('' if finetune==True else 'no-'))
checkpoint_filepath = config.root_path+'/models/{}/checkpoint.ckpt'.format(model_path)
log_file = '{}/predictions/{}/predictions.csv'.format(config.root_path, hash)

checkpoints.append((checkpoint_filepath, log_file))

In [30]:
hash = "jA6HL"
dataset_type = "wiki"
num_samples = 40561
augmentation = False
finetune = True

# model_path = 'LDABERT2/hybrid-conv/{}-{}-1-pct-1-aug_{}/finetune-True'.format(dataset_type, num_samples, hash)
model_path = 'LDABERT2/simple/{}-{}-1-pct-{}-aug_{}/{}finetune'.format(dataset_type, num_samples, (1 if augmentation==True else 0), hash, ('' if finetune==True else 'no-'))
checkpoint_filepath = config.root_path+'/models/{}/checkpoint.ckpt'.format(model_path)
log_file = '{}/predictions/{}/predictions.csv'.format(config.root_path, hash)

checkpoints.append((checkpoint_filepath, log_file))

In [31]:
hash = "DRh4S"
dataset_type = "fiction"
num_samples = 43721
augmentation = False

# model_path = 'LDABERT2/hybrid-conv/{}-{}-1-pct-1-aug_{}/finetune-True'.format(dataset_type, num_samples, hash)
model_path = 'LDABERT2/simple/{}-{}-1-pct-{}-aug_{}/{}finetune'.format(dataset_type, num_samples, (1 if augmentation==True else 0), hash, ('' if finetune==True else 'no-'))
checkpoint_filepath = config.root_path+'/models/{}/checkpoint.ckpt'.format(model_path)
log_file = '{}/predictions/{}/predictions.csv'.format(config.root_path, hash)

checkpoints.append((checkpoint_filepath, log_file))

In [32]:
checkpoints

[('/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/simple/clinical-24771-1-pct-0-aug_du7Ee/finetune/checkpoint.ckpt',
  '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/predictions/du7Ee/predictions.csv'),
 ('/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/simple/wiki-40561-1-pct-0-aug_jA6HL/finetune/checkpoint.ckpt',
  '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/predictions/jA6HL/predictions.csv'),
 ('/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/simple/fiction-43721-1-pct-0-aug_DRh4S/finetune/checkpoint.ckpt',
  '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/predictions/DRh4S/predictions.csv')]

## Inference

In [ ]:
import csv

slice_size = 125
lda_gamma = 15

# for threshold in [0.25, 0.5, 0.75, 0.8]:
for checkpoint in checkpoints:
  predictions_log = []
  checkpoint_filepath = checkpoint[0]
  log_file = checkpoint[1]

  model = ContextEncoderSimple(final_dropout=0.5,
                              dense_neurons=128,
                              max_sentence_length=128,
                              bert_trainable=False,
                              gamma=lda_gamma)

  model.load_weights(checkpoint_filepath)

  for i in range(0, testing_saved_vectors.shape[0]//slice_size):
    start = i*slice_size
    end = start + slice_size
    print(start, end)

    sliced_tokenized_sentences = {
        'input_ids': testing_saved_tokenized_sentences['input_ids'][start:end],
        'token_type_ids': testing_saved_tokenized_sentences['token_type_ids'][start:end],
        'attention_mask': testing_saved_tokenized_sentences['attention_mask'][start:end]
    }
    sliced_vectors = testing_saved_vectors[start:end]

    left_input, mid_input, right_input = testing_dataset.format_sentences_tri_input_plus(sliced_tokenized_sentences)
    lda_left_input, lda_mid_input, lda_right_input = testing_dataset.format_sentences_tri_input(sliced_vectors)

    logits = model([
        left_input,
        mid_input,
        right_input,
        lda_left_input,
        lda_mid_input,
        lda_right_input
    ])

    logits_flattened = [float(p) for p in logits]

    pred_threshold = 0.5
    predictions = [1 if float(logit) >= pred_threshold else 0 for logit in logits_flattened]

    k = 14

    string_predictions = "".join([str(i) for i in predictions])
    string_ground_truth = "".join([str(i) for i in testing_saved_labels[start:end]])
    print(string_predictions)
    print(string_ground_truth)
    overall_windowdiff = windowdiff(string_predictions, string_ground_truth, k)
    overall_pk = pk(string_predictions, string_ground_truth, k)

    for pred_idx, pred in enumerate(predictions):
      log = {
          "logit": logits_flattened[pred_idx],
          "prediction": pred,
          "ground_truth": testing_saved_labels[start:end][pred_idx],
          "text": testing_saved_sentences[start:end][pred_idx],
          "lda_gamma": lda_gamma,
          "prediction_threshold": pred_threshold,
      }
      predictions_log.append(log)
      # print(log)

    print("{},{},{},{}".format(overall_windowdiff, overall_pk, k, k))

  try:
    if not os.path.exists(os.path.dirname(log_file)):
        os.makedirs(os.path.dirname(log_file))
    mode = 'a' if os.path.exists(log_file) else 'w'
    with open(log_file, mode) as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["prediction", "ground_truth", "text", "lda_gamma", "prediction_threshold", "logit"])
        writer.writeheader()
        for data in predictions_log:
            writer.writerow(data)
  except IOError:
      print("I/O error")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


0 125
00000010000000011010001111101111100001000010011000001100010101000001000010001000000010000000100000100101000111110001010001001
10000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000100000000000000000000
0.9910714285714286,0.7410714285714286,14,14
125 250
00100111100001100000000000000000000000000110000011001100000000000000101000011100010110111001010001100001000011110100111110100
00000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0.9732142857142857,0.9642857142857143,14,14
250 375
00001100010000000000000110000001010000000000000000001010001000100001011000001001111100000001000110000101011100110100000101110
00100000000000000000000000000000000000000000000000000000000010000000000000000000000001000000000000001000010000000000000000000
0.9553571428571429,0.5089285714285714,14,14
375 500
0000100010000001000000000000111111110010000000011000110100000100101000010011111000

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


0 125
00011101011011000000000100000100000001100101110001000000000011111011100101011000000000000101000001010111000000001000000000000
10000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000100000000000000000000
0.9910714285714286,0.7410714285714286,14,14
125 250
00001001000001000001100000000110011100100010011001001000011100101111101110000010100101111101001100010100001111110011111000111
00000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
1.0,0.875,14,14
250 375
11011111000001100100000001000000010100000111000110101010011000000000010000001000010000011100001111101110101101111111010110111
00100000000000000000000000000000000000000000000000000000000010000000000000000000000001000000000000001000010000000000000000000
0.9821428571428571,0.5535714285714286,14,14
375 500
11100010101000000101000010010011001010111000100111111101010000000111100000101110111110110001100001011110100111

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


0 125
10000000000000000000000100000000010000000000010000000011000000000100000000000000010000100000000000000000001000000000010000100
10000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000100000000000000000000
0.6696428571428571,0.625,14,14
125 250
00000000000000000000000000000100000000000000000000100000000000000000000000001000000000000000000000000000000000001000000000000
00000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0.36607142857142855,0.36607142857142855,14,14
250 375
00000000000000000001000110000000000000000000000000000001000000001011001101000000101000000000000000000000000000000000000000000
00100000000000000000000000000000000000000000000000000000000010000000000000000000000001000000000000001000010000000000000000000
0.7053571428571429,0.5357142857142857,14,14
375 500
000000000000000010000000000000000001000000100001000000000010000010000000010000000001000000000

In [41]:
try:
  import csv
  if not os.path.exists(os.path.dirname(log_file)):
      os.makedirs(os.path.dirname(log_file))
  mode = 'a' if os.path.exists(log_file) else 'w'
  with open(log_file, mode) as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=["prediction", "ground_truth", "text", "lda_gamma", "prediction_threshold", "logit"])
      writer.writeheader()
      for data in predictions_log:
          writer.writerow(data)
except IOError:
    print("I/O error")